# Exercice C



# Question A



### Question 1

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB


import warnings
warnings.filterwarnings("ignore")

In [0]:
df_train = pd.read_csv('titanic_train.csv')
df_test = pd.read_csv('titanic_test.csv')
def pourcentageDf(df):
    return df.value_counts()/df.shape[0]*100

In [0]:
pourcentageClass1 = pourcentageDf(df_train[df_train['Pclass']==1]['Survived'])
pourcentageClass2 = pourcentageDf(df_train[df_train['Pclass']==2]['Survived'])
pourcentageClass3 = pourcentageDf(df_train[df_train['Pclass']==3]['Survived'])

result = pd.concat([pourcentageClass1, pourcentageClass2, pourcentageClass3],
                   axis=1)
result.columns = ['Classe 1','Classe 2','Classe 3']
result

,Classe 1,Classe 2,Classe 3
0,31.168831,50.0,78.412698
1,68.831169,50.0,21.587302


### Question 2

In [0]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()

df_train_labeled = df_train

df_train_labeled = df_train_labeled.drop(['PassengerId','Name','FullName','Ticket','Cabin'],axis=1)


df_train_labeled.iloc[:,2:3] = enc.fit_transform(df_train.iloc[:,2:3])
df_train_labeled.iloc[:,7:8] = enc.fit_transform(df_train.iloc[:,7:8].astype(str))



def fare2AttBuilder(df):
 
  fare_cat = np.zeros(df.shape[0])
  
  for i in range(len(fare_cat)):
    
    if df['Fare'][i]<10:
      fare_cat[i]=1
      
    elif df['Fare'][i]<20 and df['Fare'][i]>10:
       fare_cat[i]=2
        
    elif df['Fare'][i]>20 and df['Fare'][i]<30:
       fare_cat[i]=3
  
    else :
       fare_cat[i]=4
        
  dffare_cat = pd.DataFrame(fare_cat)
  dffare_cat.columns=['Fare2']
  df = pd.concat([df, dffare_cat], axis=1)

  return (df)


df_train_labeled = fare2AttBuilder(df_train_labeled)
df_train_labeled.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Fare2
0,1,1,0,NaN,0,0,30.5000,0,4.0
1,1,1,0,50.0,0,1,247.5208,0,4.0
2,1,3,2,NaN,0,0,7.7500,0,1.0
3,1,3,2,NaN,2,0,23.2500,2,3.0
4,0,3,2,19.0,0,0,0.0000,0,1.0


### Question 3 

In [0]:
from scipy.stats import chisquare
chisquare(df_train_labeled['Pclass'], df_train_labeled['Fare2'])

Power_divergenceResult(statistic=1232.1666666666665, pvalue=3.540770184018379e-47)

On peut observer de manière évidente que la classe correspondant au prix du billet àune forte influence sur le resultat
Ainsi valait-il mieux avoir acheté un billet au prix fort au moment où le bateau allait sombrer.

### Question 4

In [0]:
pourcentageFare1 = pourcentageDf(df_train_labeled[df_train_labeled['Fare2']==1]['Survived'])
pourcentageFare2 = pourcentageDf(df_train_labeled[df_train_labeled['Fare2']==2]['Survived'])
pourcentageFare3 = pourcentageDf(df_train_labeled[df_train_labeled['Fare2']==3]['Survived'])
pourcentageFare4 = pourcentageDf(df_train_labeled[df_train_labeled['Fare2']==4]['Survived'])

result = pd.concat([pourcentageFare1, pourcentageFare2, pourcentageFare3,pourcentageFare4],
                   axis=1)
result.columns = ['Fare 1','Fare 2','Fare 3','Fare 4']
result

,Fare 1,Fare 2,Fare 3,Fare 4
0,82.959641,56.140351,55.555556,34.939759
1,17.040359,43.859649,44.444444,65.060241


On observe de fortes dispartiés entre le pourcentage de survie et le prix du billet acheté par l'individu, les individus ayant achetés les billets les moins chers (<10) sont seulement 17% a avoir survécu, ceux des classes moyennes (fare2 & fare 3) près de 44% et 65 % pour les billets les plus chers, on retrouve donc des résultats assez proches de ceux observés à la question précédente avec les différentes classes, ce qui est cohérent avec le fait que les classes et le tarif des billets soient corrélés.

### Question 5

In [0]:
enc = LabelEncoder()

df_test_labeled = df_test

df_test_labeled = df_test_labeled.drop(['PassengerId','Name','FullName','Ticket','Cabin'],axis=1)


df_test_labeled.iloc[:,2:3] = enc.fit_transform(df_test_labeled.iloc[:,2:3])
df_test_labeled.iloc[:,7:8] = enc.fit_transform(df_test_labeled.iloc[:,7:8].astype(str))


df_test_labeled = fare2AttBuilder(df_test_labeled)


gnbModelB=GaussianNB()
gnbModel.fit(df_train_labeled[['Pclass','Fare2']],df_train_labeled['Survived'])

expected = df_test_labeled['Survived']
predicted = gnbModel.predict(df_test_labeled[['Pclass','Fare2']])

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       0.70      0.77      0.73       192
          1       0.49      0.40      0.44       106

avg / total       0.62      0.64      0.63       298

[[148  44]
 [ 64  42]]


**Conclusion :** 


La précision de notre prédiction est ici de 62% soit bien inférerieur avec notre prédicteur obtenu à la question précédente. Cela souligne la tendance observé depuis le début du tp que le sexe avait une forte influence sur la prédiction de survie d'un individu.